In [1]:
# Biblioteca de imports
import numpy as np
import pandas as pd
import string


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer

In [ ]:
# Baixando o dataset
go_emotion_df = pd.read_csv('data/go_emotions_dataset.csv')

In [ ]:
# Crating a function to clean the text
def cleaning(text):
    text = text.lower()
    text = text.replace('’','')
    text = text.strip()
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '') 
        
    text = ''.join(char for char in text if not char.isdigit())
    
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    tokenized_text = [word for word in word_tokens if not word in stop_words]
    
    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v") 
        for word in tokenized_text
    ]
    
    cleaned_sentence = ' '.join(word for word in lemmatized)
    
    return cleaned_sentence
    

In [ ]:
# Creating new feature for processed texts
go_emotion_df['text_processed'] = ''go_emotion_df['text'].apply(cleaning)